In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + 'C:\\Users\\rdpuser\\Documents\\Github\\parkit;'
sys.path.append('C:\\Users\\rdpuser\\Documents\\Github\\parkit')

In [3]:
import logging
import multiprocessing
import queue

import parkit as pk

logger = logging.getLogger(__name__)

pk.import_site('C:\\Users\\rdpuser\\Desktop\\test')

In [4]:
# Queue

In [5]:
q1 = queue.Queue()
q2 = pk.Queue('memory')
q3 = multiprocessing.Queue()
q4 = pk.Queue()

In [6]:
%%timeit
for i in range(1000):
    q1.put_nowait(i)
for i in range(1000):
    q1.get_nowait()

5.02 ms ± 181 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
with pk.transaction(q2):
    for i in range(1000):
        q2.put(i)
    for i in range(1000):
        q2.get()

11.7 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit
for i in range(1000):
    q2.put(i)
for i in range(1000):
    q2.get()

92.3 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
for i in range(1000):
    q3.put(i)
for i in range(1000):
    q3.get()

39.5 ms ± 950 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
# Array

In [11]:
a1 = list()
a2 = pk.Array('memory')
a3 = multiprocessing.Array('i', range(10000))

In [12]:
%%timeit
for i in range(10000):
    a1.append(i)
a1.clear()

953 µs ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%%timeit
with pk.transaction(a2):
    for i in range(10000):
        a2.append(i)
a2.clear()

56.5 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit
for i in range(10000):
    a3[i] = i

23.4 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
# Dict

In [16]:
d1 = dict()
d2 = pk.Dict('memory')
d3 = multiprocessing.Manager().dict()

In [17]:
data = []
for i in range(10000):
    data.append((i, i))

In [18]:
%%timeit
for i in range(10000):
    d1[i] = i
d1.clear()

1.2 ms ± 44.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
%%timeit
d1.update(data)
d1.clear()

516 µs ± 8.37 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
%%timeit
with pk.transaction(d2):
    for i in range(10000):
        d2[i] = i
    d2.clear()

53.9 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%%timeit
d2.update(data)
d2.clear()

26.7 ms ± 993 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%%timeit
for i in range(10000):
    d2[i] = i
d2.clear()

464 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
for i in range(10000):
    d3[i] = i
d3.clear()

834 ms ± 19.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
d3.update(data)
d3.clear()

4.48 ms ± 258 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
# File

In [26]:
import pandas as pd
import numpy as np

In [27]:
rows = []
for i in range(1000000):
    rows.append(dict(a = i, b = i))
df = pd.DataFrame(rows)

In [28]:
%%timeit
df.to_feather('test')

29.7 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
f = pk.File('memory', mode = 'wb')

In [30]:
%%timeit
with f:
    df.to_feather(f)

73.8 ms ± 3.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
%%timeit
_ = pd.read_feather('test')

24.6 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%%timeit
f.mode = 'rb'
with f:
    assert df.equals(pd.read_feather(f))

29 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%timeit
f.mode = 'rb'
with pk.snapshot(f):
    assert isinstance(f.content, memoryview)

82.5 µs ± 2.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [34]:
a = np.zeros((1000,1000))

In [35]:
f = pk.File(mode = 'wb')
with f:
    np.save(f, a)

TransactionError: 

In [ ]:
%%timeit
f.mode = 'rb'
with pk.snapshot(f):
    assert isinstance(f.content, memoryview)